# Flight Data Clean Up

In [1]:
# Import Dependencies
import pandas as pd
from pathlib import Path
import glob
import zipfile
import os
import shutil
import re

## Extract and Load Flight Information

In [2]:
# List of all source zips
zips = ['Resources/BTS/BTS_Q1_2023.zip','Resources/BTS/BTS_Q2_2023.zip',
       'Resources/BTS/BTS_Q3_2023.zip','Resources/BTS/BTS_Q4_2023.zip']

# Directory to extract to, confirm it exists
extraction_path = 'Resources/BTS/extracted_zips'
os.makedirs(extraction_path, exist_ok=True)

# Iterate through zips and extract all
for zip in zips:
    with zipfile.ZipFile(zip, 'r') as zip_ref:
        zip_ref.extractall(extraction_path)

    print(f"Extracted {zip} to {extraction_path}")

Extracted Resources/BTS/BTS_Q1_2023.zip to Resources/BTS/extracted_zips
Extracted Resources/BTS/BTS_Q2_2023.zip to Resources/BTS/extracted_zips
Extracted Resources/BTS/BTS_Q3_2023.zip to Resources/BTS/extracted_zips
Extracted Resources/BTS/BTS_Q4_2023.zip to Resources/BTS/extracted_zips


In [3]:
# Read all extracted CSV's
# glob is an efficient way to work through all CSV's in a directory: https://docs.python.org/3/library/glob.html
csv_files = glob.glob(f"{extraction_path}/*.csv")

# Empty list for storing DataFrames
dataframes = []

# Iterate through csv's and store into DataFrames list
for csv_file in csv_files:
    df = pd.read_csv(csv_file, low_memory=False)
    dataframes.append(df)
# Concatenante into single DataFrame
unfiltered_flight_info = pd.concat(dataframes, ignore_index = True)
pd.set_option('display.max_columns', None)
unfiltered_flight_info.head()

,Year,Quarter,Month,DayofMonth,DayOfWeek,FlightDate,Marketing_Airline_Network,Operated_or_Branded_Code_Share_Partners,DOT_ID_Marketing_Airline,IATA_Code_Marketing_Airline,Flight_Number_Marketing_Airline,Originally_Scheduled_Code_Share_Airline,DOT_ID_Originally_Scheduled_Code_Share_Airline,IATA_Code_Originally_Scheduled_Code_Share_Airline,Flight_Num_Originally_Scheduled_Code_Share_Airline,Operating_Airline,DOT_ID_Operating_Airline,IATA_Code_Operating_Airline,Tail_Number,Flight_Number_Operating_Airline,OriginAirportID,OriginAirportSeqID,OriginCityMarketID,Origin,OriginCityName,OriginState,OriginStateFips,OriginStateName,OriginWac,DestAirportID,DestAirportSeqID,DestCityMarketID,Dest,DestCityName,DestState,DestStateFips,DestStateName,DestWac,CRSDepTime,DepTime,DepDelay,DepDelayMinutes,DepDel15,DepartureDelayGroups,DepTimeBlk,TaxiOut,WheelsOff,WheelsOn,TaxiIn,CRSArrTime,ArrTime,ArrDelay,ArrDelayMinutes,ArrDel15,ArrivalDelayGroups,ArrTimeBlk,Cancelled,CancellationCode,Diverted,CRSElapsedTime,ActualElapsedTime,AirTime,Flights,Distance,DistanceGroup,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,FirstDepTime,TotalAddGTime,LongestAddGTime,DivAirportLandings,DivReachedDest,DivActualElapsedTime,DivArrDelay,DivDistance,Div1Airport,Div1AirportID,Div1AirportSeqID,Div1WheelsOn,Div1TotalGTime,Div1LongestGTime,Div1WheelsOff,Div1TailNum,Div2Airport,Div2AirportID,Div2AirportSeqID,Div2WheelsOn,Div2TotalGTime,Div2LongestGTime,Div2WheelsOff,Div2TailNum,Div3Airport,Div3AirportID,Div3AirportSeqID,Div3WheelsOn,Div3TotalGTime,Div3LongestGTime,Div3WheelsOff,Div3TailNum,Div4Airport,Div4AirportID,Div4AirportSeqID,Div4WheelsOn,Div4TotalGTime,Div4LongestGTime,Div4WheelsOff,Div4TailNum,Div5Airport,Div5AirportID,Div5AirportSeqID,Div5WheelsOn,Div5TotalGTime,Div5LongestGTime,Div5WheelsOff,Div5TailNum,Duplicate,Unnamed: 119
0,2023,1,1,22,7,2023-01-22,B6,B6,20409,B6,1447,NaN,NaN,NaN,NaN,B6,20409,B6,N636JB,1447,12197,1219702,31703,HPN,"White Plains, NY",NY,36,New York,22,15304,1530402,33195,TPA,"Tampa, FL",FL,12,Florida,33,800,756.0,-4.0,0.0,0.0,-1.0,0800-0859,12.0,808.0,1053.0,5.0,1102,1058.0,-4.0,0.0,0.0,-1.0,1100-1159,0.0,NaN,0.0,182.0,182.0,165.0,1.0,1032.0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN
1,2023,1,1,22,7,2023-01-22,B6,B6,20409,B6,1451,NaN,NaN,NaN,NaN,B6,20409,B6,N3112J,1451,10721,1072102,30721,BOS,"Boston, MA",MA,25,Massachusetts,13,13204,1320402,31454,MCO,"Orlando, FL",FL,12,Florida,33,1955,2048.0,53.0,53.0,1.0,3.0,1900-1959,16.0,2104.0,6.0,10.0,2322,16.0,54.0,54.0,1.0,3.0,2300-2359,0.0,NaN,0.0,207.0,208.0,182.0,1.0,1121.0,5,39.0,0.0,1.0,0.0,14.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN
2,2023,1,1,22,7,2023-01-22,B6,B6,20409,B6,1453,NaN,NaN,NaN,NaN,B6,20409,B6,N507JT,1453,11697,1169706,32467,FLL,"Fort Lauderdale, FL",FL,12,Florida,33,14843,1484306,34819,SJU,"San Juan, PR",PR,72,Puerto Rico,3,1727,1734.0,7.0,7.0,0.0,0.0,1700-1759,25.0,1759.0,2110.0,7.0,2054,2117.0,23.0,23.0,1.0,1.0,2000-2059,0.0,NaN,0.0,147.0,163.0,131.0,1.0,1046.0,5,7.0,0.0,16.0,0.0,0.0,1713.0,6.0,6.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN
3,2023,1,1,22,7,2023-01-22,B6,B6,20409,B6,1454,NaN,NaN,NaN,NaN,B6,20409,B6,N956JT,1454,14843,1484306,34819,SJU,"San Juan, PR",PR,72,Puerto Rico,3,11697,1169706,32467,FLL,"Fort Lauderdale, FL",FL,12,Florida,33,1140,1147.0,7.0,7.0,0.0,0.0,1100-1159,15.0,1202.0,1323.0,5.0,1328,1328.0,0.0,0.0,0.0,0.0,1300-1359,0.0,NaN,0.0,168.0,161.0,141.0,1.0,1046.0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [4]:
# Specify columns and remove limitation on display
pd.set_option('display.max_columns', None)
flights_df = pd.DataFrame(unfiltered_flight_info[['FlightDate','IATA_Code_Marketing_Airline', 
                                                  'Operated_or_Branded_Code_Share_Partners',
                                                  'Operating_Airline ','Flight_Number_Marketing_Airline',
                                                  'Origin','Dest','DepDelay','DepDelayMinutes',
                                                  'DepDel15','ArrDelay','ArrDelayMinutes','Cancelled',
                                                  'Diverted','CarrierDelay','WeatherDelay','NASDelay',
                                                  'SecurityDelay','LateAircraftDelay']])
flights_df.head()

,FlightDate,IATA_Code_Marketing_Airline,Operated_or_Branded_Code_Share_Partners,Operating_Airline,Flight_Number_Marketing_Airline,Origin,Dest,DepDelay,DepDelayMinutes,DepDel15,ArrDelay,ArrDelayMinutes,Cancelled,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
0,2023-01-22,B6,B6,B6,1447,HPN,TPA,-4.0,0.0,0.0,-4.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
1,2023-01-22,B6,B6,B6,1451,BOS,MCO,53.0,53.0,1.0,54.0,54.0,0.0,0.0,39.0,0.0,1.0,0.0,14.0
2,2023-01-22,B6,B6,B6,1453,FLL,SJU,7.0,7.0,0.0,23.0,23.0,0.0,0.0,7.0,0.0,16.0,0.0,0.0
3,2023-01-22,B6,B6,B6,1454,SJU,FLL,7.0,7.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
4,2023-01-22,B6,B6,B6,1455,BOS,DCA,-8.0,0.0,0.0,-17.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN


In [5]:
# Empty and delete the extraction_path directory
try:
    shutil.rmtree(extraction_path)
    print(f"Directory {extraction_path} and all its contents have been deleted.")
except OSError as e:
    print(f"Error: {e.strerror}")

Directory Resources/BTS/extracted_zips and all its contents have been deleted.


## Airport Selection

In [6]:
# number of unique origin airports in data
origins_df = unfiltered_flight_info[['Origin','OriginCityName']]
origins_df['Origin'].nunique()

359

In [7]:
# number of unique destination airports in data
destinations_df = unfiltered_flight_info[['Dest','DestCityName']]
destinations_df['Dest'].nunique()

359

In [8]:
# total number of unique airports in data
# compare the origin airports to destination airports
ori_match = origins_df['Origin'].isin(destinations_df['Dest'])
dest_match = destinations_df['Dest'].isin(origins_df['Origin'])
ori_dest_match = ori_match | dest_match

# only keep rows for airports that are listed in both columns
flight_airports = flights_df[ori_dest_match]
flight_airports['Origin'].nunique()
# confirmed that the same 359 airports have travel data

359

## Above 3 cells combined and simplified below

In [30]:
# Determine total number of unique airports in flight data
origins = unfiltered_flight_info['Origin']
destinations = unfiltered_flight_info['Dest']
# compare the origin airports to destination airports
ori_match = origins.isin(destinations)
dest_match = destinations.isin(origins)
ori_dest_match = ori_match | dest_match

# only keep rows for airports that have inputs in both columns
flight_airports = flights_df[ori_dest_match]
# show number of unique airports that have travel data
flight_airports['Origin'].nunique()

359

In [9]:
#  compare us_airport data to airports in flight data
# Read in CSV of us-airports
us_airports = pd.read_csv('Resources/us-airports.csv')
# create exclusion
iata_match = us_airports['iata_code'].isin(flight_airports['Origin'])
# filter the data
us_airports_filtered = us_airports[iata_match].reset_index()
# count number of airports
us_airports_filtered['iata_code'].nunique()

351

In [10]:
# Which 8 airports do we have flight data for that our airport list doesn't have?
# create mask for inclusion
iata_match = flight_airports['Origin'].isin(us_airports_filtered['iata_code'])
# drop data that doesn't match 
us_airports_missing = flight_airports[~iata_match]
# show the extra airport codes in flight data
exclude_airports = us_airports_missing['Origin'].unique()
exclude_airports

array(['SJU', 'STT', 'BQN', 'PSE', 'STX', 'PPG', 'GUM', 'SPN'],
      dtype=object)

Researched IATA codes to determine location of airports(Google):
>SJU, BQM, PSE = Puerto Rico

>STT,STX, = U.S. Virgin Islands

>PPG = American Samoa (unincorporated territory)

>GUM = Guam

>SPN = Northern Mariana Islands

Conclusion: These 8 airports can be eliminated from flight data to keep it to 50 states.

Total airports: 351

In [11]:
# Excluding data from the 8 erroneous airports 
exclusions = flights_df['Origin'].isin(us_airports_missing['Origin'])

filtered_flights = flights_df[~exclusions]
filtered_flights['Origin'].nunique()

351

In [12]:
# Filter airports to just those that have flight data
airport_mask = us_airports['iata_code'].isin(filtered_flights['Origin'])

us_airports_filtered = pd.DataFrame(us_airports[airport_mask])
us_airports_filtered['iata_code'].nunique()

351

In [13]:
us_airports.head()

,id,ident,type,name,latitude_deg,longitude_deg,elevation_ft,continent,country_name,iso_country,region_name,iso_region,local_region,municipality,scheduled_service,gps_code,iata_code,local_code,home_link,wikipedia_link,keywords,score,last_updated
0,#meta +id,#meta +code,#loc +airport +type,#loc +airport +name,#geo +lat,#geo +lon,#geo +elevation +ft,#region +continent +code,#country +name,#country +code +iso2,#adm1 +name,#adm1 +code +iso,#adm1 +code +local,#loc +municipality +name,#status +scheduled,#loc +airport +code +gps,#loc +airport +code +iata,#loc +airport +code +local,#meta +url +airport,#meta +url +wikipedia,#meta +keywords,#meta +score,#date +updated
1,3632,KLAX,large_airport,Los Angeles International Airport,33.942501,-118.407997,125,NaN,United States,US,California,US-CA,CA,Los Angeles,1,KLAX,LAX,LAX,https://www.flylax.com/,https://en.wikipedia.org/wiki/Los_Angeles_Inte...,Tom Bradley,1335475,2023-12-21T12:31:02+00:00
2,3754,KORD,large_airport,Chicago O'Hare International Airport,41.9786,-87.9048,680,NaN,United States,US,Illinois,US-IL,IL,Chicago,1,KORD,ORD,ORD,https://www.flychicago.com/ohare/home/pages/de...,https://en.wikipedia.org/wiki/O'Hare_Internati...,"CHI, Orchard Place",1503175,2024-03-09T23:28:49+00:00
3,3622,KJFK,large_airport,John F Kennedy International Airport,40.639447,-73.779317,13,NaN,United States,US,New York,US-NY,NY,New York,1,KJFK,JFK,JFK,https://www.jfkairport.com/,https://en.wikipedia.org/wiki/John_F._Kennedy_...,"Manhattan, New York City, NYC, Idlewild, IDL, ...",1052075,2022-10-18T18:49:55+00:00
4,3384,KATL,large_airport,Hartsfield Jackson Atlanta International Airport,33.6367,-84.428101,1026,NaN,United States,US,Georgia,US-GA,GA,Atlanta,1,KATL,ATL,ATL,http://www.atlanta-airport.com/,https://en.wikipedia.org/wiki/Hartsfield–Jacks...,NaN,2002475,2018-09-19T14:50:01+00:00


In [14]:
us_airports_filtered.head()

,id,ident,type,name,latitude_deg,longitude_deg,elevation_ft,continent,country_name,iso_country,region_name,iso_region,local_region,municipality,scheduled_service,gps_code,iata_code,local_code,home_link,wikipedia_link,keywords,score,last_updated
1,3632,KLAX,large_airport,Los Angeles International Airport,33.942501,-118.407997,125,NaN,United States,US,California,US-CA,CA,Los Angeles,1,KLAX,LAX,LAX,https://www.flylax.com/,https://en.wikipedia.org/wiki/Los_Angeles_Inte...,Tom Bradley,1335475,2023-12-21T12:31:02+00:00
2,3754,KORD,large_airport,Chicago O'Hare International Airport,41.9786,-87.9048,680,NaN,United States,US,Illinois,US-IL,IL,Chicago,1,KORD,ORD,ORD,https://www.flychicago.com/ohare/home/pages/de...,https://en.wikipedia.org/wiki/O'Hare_Internati...,"CHI, Orchard Place",1503175,2024-03-09T23:28:49+00:00
3,3622,KJFK,large_airport,John F Kennedy International Airport,40.639447,-73.779317,13,NaN,United States,US,New York,US-NY,NY,New York,1,KJFK,JFK,JFK,https://www.jfkairport.com/,https://en.wikipedia.org/wiki/John_F._Kennedy_...,"Manhattan, New York City, NYC, Idlewild, IDL, ...",1052075,2022-10-18T18:49:55+00:00
4,3384,KATL,large_airport,Hartsfield Jackson Atlanta International Airport,33.6367,-84.428101,1026,NaN,United States,US,Georgia,US-GA,GA,Atlanta,1,KATL,ATL,ATL,http://www.atlanta-airport.com/,https://en.wikipedia.org/wiki/Hartsfield–Jacks...,NaN,2002475,2018-09-19T14:50:01+00:00
5,3878,KSFO,large_airport,San Francisco International Airport,37.61899948120117,-122.375,13,NaN,United States,US,California,US-CA,CA,San Francisco,1,KSFO,SFO,SFO,http://www.flysfo.com/,https://en.wikipedia.org/wiki/San_Francisco_In...,"QSF, QBA",1112475,2008-06-13T14:30:04+00:00


In [15]:
us_airports_filtered['type'].value_counts()

type
medium_airport    276
large_airport      65
small_airport      10
Name: count, dtype: int64

In [16]:
# Export filtered data sets to CSV
us_airports_filtered.to_csv('Resources/us_airports_filtered.csv')
filtered_flights.to_csv('Resources/filtered_flights.csv')

In [17]:
# # Zip filtered_flights.csv and delete to allow github upload
csv_path = 'Resources/filtered_flights.csv'
zip_path = 'Resources/filtered_flights.zip'

with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
    zipf.write(csv_path, os.path.basename(csv_path))

# Check if the ZIP file exists and the size condition is met
if os.path.exists(zip_path):
    # Get the size of the ZIP file in bytes
    zip_size = os.path.getsize(zip_path)
    
    # Define the size condition, e.g., file must be larger than 0 bytes to ensure it's not empty
    if zip_size > 50:
        os.remove(csv_path)
        print(f"The file {csv_path} has been zipped, is not empty, and has been deleted.")
    else:
        print(f"The file {zip_path} is empty.")
else:
    print("Error: The zip file was not created successfully.")

The file Resources/filtered_flights.csv has been zipped, is not empty, and has been deleted.


## Identify Airlines

In [18]:
# Identify unique IATA codes and store in a list in order of frequency
airline_IATA_codes = filtered_flights['IATA_Code_Marketing_Airline'].value_counts().index.to_list()
airline_IATA_codes

['AA', 'DL', 'WN', 'UA', 'AS', 'B6', 'NK', 'F9', 'G4', 'HA']

In [19]:
# Identify airlines and make list in matching order 
# Source: https://www.iata.org/en/publications/directories/code-search/
airline_names = ['American Airlines','Delta Airlines','Southwest Airlines','United Airlines',
                 'Alaska Airlines','JetBlue','Spirit Airlines','Frontier Airlines','Allegiant Air',
                 'Hawaiian Airlines']
# Build dictionary of IATA_codes and airline names to turn into a data frame
airline_data = {'airline_IATA_code':airline_IATA_codes,
               'airline_name':airline_names}

In [20]:
# create dataframe
airline_df = pd.DataFrame(airline_data)
airline_df

,airline_IATA_code,airline_name
0,AA,American Airlines
1,DL,Delta Airlines
2,WN,Southwest Airlines
3,UA,United Airlines
4,AS,Alaska Airlines
5,B6,JetBlue
6,NK,Spirit Airlines
7,F9,Frontier Airlines
8,G4,Allegiant Air
9,HA,Hawaiian Airlines


In [21]:
# Export to CSV
airline_df.to_csv('Resources/airlines.csv')

## Get city of airport from the flight data and insert into airport information for SQL

In [22]:
us_airports_filtered.head(1)

,id,ident,type,name,latitude_deg,longitude_deg,elevation_ft,continent,country_name,iso_country,region_name,iso_region,local_region,municipality,scheduled_service,gps_code,iata_code,local_code,home_link,wikipedia_link,keywords,score,last_updated
1,3632,KLAX,large_airport,Los Angeles International Airport,33.942501,-118.407997,125,NaN,United States,US,California,US-CA,CA,Los Angeles,1,KLAX,LAX,LAX,https://www.flylax.com/,https://en.wikipedia.org/wiki/Los_Angeles_Inte...,Tom Bradley,1335475,2023-12-21T12:31:02+00:00


In [23]:
# filter destination airports to unique entries
dest_filtered = destinations_df.drop_duplicates(subset=['Dest', 'DestCityName'])
dest_filtered.count()

Dest            359
DestCityName    359
dtype: int64

In [24]:
# check if there are any DestCityName rows that don't conform to City, State format
# regex = special characters include anything not a letter, number, comma, or whitespace
regex = r'^[A-Za-z0-9, ]+$'
# Dictionary to store results
not_uniform = []
# iterate through each row and check if cell contains comma AND doesn't match regex special char
for index, row in dest_filtered.iterrows():
    cell = row['DestCityName']
    if ',' in cell and not re.match(regex, cell):
        not_uniform.append(cell)

print(not_uniform)

# Learned: Even if there are slashes for metro areas, the format for ', state' still same

['West Palm Beach/Palm Beach, FL', 'Raleigh/Durham, NC', 'Sarasota/Bradenton, FL', 'Dallas/Fort Worth, TX', 'Montrose/Delta, CO', 'Bismarck/Mandan, ND', 'St. Louis, MO', 'Gulfport/Biloxi, MS', 'Jackson/Vicksburg, MS', 'Greensboro/High Point, NC', 'Cedar Rapids/Iowa City, IA', 'Jacksonville/Camp Lejeune, NC', 'Pasco/Kennewick/Richland, WA', 'Newburgh/Poughkeepsie, NY', 'Bloomington/Normal, IL', 'Allentown/Bethlehem/Easton, PA', 'St. Petersburg, FL', 'Bristol/Johnson City/Kingsport, TN', 'Elmira/Corning, NY', 'Mission/McAllen/Edinburg, TX', 'St. Cloud, MN', 'Clarksburg/Fairmont, WV', 'Manhattan/Ft. Riley, KS', 'Midland/Odessa, TX', 'Harlingen/San Benito, TX', 'Champaign/Urbana, IL', 'Scranton/Wilkes-Barre, PA', 'Charleston/Dunbar, WV', 'Ithaca/Cortland, NY', 'Saginaw/Bay City/Midland, MI', 'Newport News/Williamsburg, VA', 'New Bern/Morehead/Beaufort, NC', 'Lawton/Fort Sill, OK', 'Bend/Redmond, OR', 'Sun Valley/Hailey/Ketchum, ID', 'Beaumont/Port Arthur, TX', 'College Station/Bryan, TX', 

In [25]:
# Create airports DF with cities
# merge airports data with city data based on IATA code
airports_cities = us_airports_filtered.merge(dest_filtered, left_on='iata_code', 
                                            right_on='Dest', how ='inner')
# Get rid of the state abbreviation from cities column
airports_cities['city_region'] = airports_cities['DestCityName'].str.split(',').str[0]
# Get rid of _airport from 'type' column
airports_cities['airport_size'] = airports_cities['type'].str.split('_').str[0]
#reset index
airports_cities.reset_index(drop=True).head()
# select columns
airports_cities.head()

,id,ident,type,name,latitude_deg,longitude_deg,elevation_ft,continent,country_name,iso_country,region_name,iso_region,local_region,municipality,scheduled_service,gps_code,iata_code,local_code,home_link,wikipedia_link,keywords,score,last_updated,Dest,DestCityName,city_region,airport_size
0,3632,KLAX,large_airport,Los Angeles International Airport,33.942501,-118.407997,125,NaN,United States,US,California,US-CA,CA,Los Angeles,1,KLAX,LAX,LAX,https://www.flylax.com/,https://en.wikipedia.org/wiki/Los_Angeles_Inte...,Tom Bradley,1335475,2023-12-21T12:31:02+00:00,LAX,"Los Angeles, CA",Los Angeles,large
1,3754,KORD,large_airport,Chicago O'Hare International Airport,41.9786,-87.9048,680,NaN,United States,US,Illinois,US-IL,IL,Chicago,1,KORD,ORD,ORD,https://www.flychicago.com/ohare/home/pages/de...,https://en.wikipedia.org/wiki/O'Hare_Internati...,"CHI, Orchard Place",1503175,2024-03-09T23:28:49+00:00,ORD,"Chicago, IL",Chicago,large
2,3622,KJFK,large_airport,John F Kennedy International Airport,40.639447,-73.779317,13,NaN,United States,US,New York,US-NY,NY,New York,1,KJFK,JFK,JFK,https://www.jfkairport.com/,https://en.wikipedia.org/wiki/John_F._Kennedy_...,"Manhattan, New York City, NYC, Idlewild, IDL, ...",1052075,2022-10-18T18:49:55+00:00,JFK,"New York, NY",New York,large
3,3384,KATL,large_airport,Hartsfield Jackson Atlanta International Airport,33.6367,-84.428101,1026,NaN,United States,US,Georgia,US-GA,GA,Atlanta,1,KATL,ATL,ATL,http://www.atlanta-airport.com/,https://en.wikipedia.org/wiki/Hartsfield–Jacks...,NaN,2002475,2018-09-19T14:50:01+00:00,ATL,"Atlanta, GA",Atlanta,large
4,3878,KSFO,large_airport,San Francisco International Airport,37.61899948120117,-122.375,13,NaN,United States,US,California,US-CA,CA,San Francisco,1,KSFO,SFO,SFO,http://www.flysfo.com/,https://en.wikipedia.org/wiki/San_Francisco_In...,"QSF, QBA",1112475,2008-06-13T14:30:04+00:00,SFO,"San Francisco, CA",San Francisco,large


In [26]:
print(airports_cities['local_region'].unique())
print(airports_cities['region_name'].unique())

['CA' 'IL' 'NY' 'GA' 'NJ' 'TX' 'NV' 'FL' 'CO' 'VA' 'AZ' 'WA' 'MA' 'PA'
 'NC' 'DC' 'MI' 'MN' 'UT' 'MD' 'MO' 'HI' 'OR' 'LA' 'TN' 'KY' 'OH' 'IN'
 'NM' 'WI' 'CT' 'AK' 'OK' 'NE' 'RI' 'SC' 'AL' 'ID' 'KS' 'ME' 'IA' 'AR'
 'NH' 'VT' 'SD' 'WY' 'MT' 'ND' 'MS' 'WV']
['California' 'Illinois' 'New York' 'Georgia' 'New Jersey' 'Texas'
 'Nevada' 'Florida' 'Colorado' 'Virginia' 'Arizona' 'Washington'
 'Massachusetts' 'Pennsylvania' 'North Carolina' 'District of Columbia'
 'Michigan' 'Minnesota' 'Utah' 'Maryland' 'Missouri' 'Hawaii' 'Oregon'
 'Louisiana' 'Tennessee' 'Kentucky' 'Ohio' 'Indiana' 'New Mexico'
 'Wisconsin' 'Connecticut' 'Alaska' 'Oklahoma' 'Nebraska' 'Rhode Island'
 'South Carolina' 'Alabama' 'Idaho' 'Kansas' 'Maine' 'Iowa' 'Arkansas'
 'New Hampshire' 'Vermont' 'South Dakota' 'Wyoming' 'Montana'
 'North Dakota' 'Mississippi' 'West Virginia']


In [27]:
# drop and rename columns
airports_drop = airports_cities.drop(columns=['Dest','DestCityName','type'])
airports_df = airports_drop.rename(columns={'name':'airport_name',
                                               'region_name':'state',
                                              'local_region':'state_abbrv'})
airports_df.head()

,id,ident,airport_name,latitude_deg,longitude_deg,elevation_ft,continent,country_name,iso_country,state,iso_region,state_abbrv,municipality,scheduled_service,gps_code,iata_code,local_code,home_link,wikipedia_link,keywords,score,last_updated,city_region,airport_size
0,3632,KLAX,Los Angeles International Airport,33.942501,-118.407997,125,NaN,United States,US,California,US-CA,CA,Los Angeles,1,KLAX,LAX,LAX,https://www.flylax.com/,https://en.wikipedia.org/wiki/Los_Angeles_Inte...,Tom Bradley,1335475,2023-12-21T12:31:02+00:00,Los Angeles,large
1,3754,KORD,Chicago O'Hare International Airport,41.9786,-87.9048,680,NaN,United States,US,Illinois,US-IL,IL,Chicago,1,KORD,ORD,ORD,https://www.flychicago.com/ohare/home/pages/de...,https://en.wikipedia.org/wiki/O'Hare_Internati...,"CHI, Orchard Place",1503175,2024-03-09T23:28:49+00:00,Chicago,large
2,3622,KJFK,John F Kennedy International Airport,40.639447,-73.779317,13,NaN,United States,US,New York,US-NY,NY,New York,1,KJFK,JFK,JFK,https://www.jfkairport.com/,https://en.wikipedia.org/wiki/John_F._Kennedy_...,"Manhattan, New York City, NYC, Idlewild, IDL, ...",1052075,2022-10-18T18:49:55+00:00,New York,large
3,3384,KATL,Hartsfield Jackson Atlanta International Airport,33.6367,-84.428101,1026,NaN,United States,US,Georgia,US-GA,GA,Atlanta,1,KATL,ATL,ATL,http://www.atlanta-airport.com/,https://en.wikipedia.org/wiki/Hartsfield–Jacks...,NaN,2002475,2018-09-19T14:50:01+00:00,Atlanta,large
4,3878,KSFO,San Francisco International Airport,37.61899948120117,-122.375,13,NaN,United States,US,California,US-CA,CA,San Francisco,1,KSFO,SFO,SFO,http://www.flysfo.com/,https://en.wikipedia.org/wiki/San_Francisco_In...,"QSF, QBA",1112475,2008-06-13T14:30:04+00:00,San Francisco,large


## Database Prep - dtypes, astype, rename

In [28]:
# Rather than small, medium, large being listed over and over - use 1,2,3 and have another table to classify

In [29]:
airports_df.dtypes

id                   object
ident                object
airport_name         object
latitude_deg         object
longitude_deg        object
elevation_ft         object
continent            object
country_name         object
iso_country          object
state                object
iso_region           object
state_abbrv          object
municipality         object
scheduled_service    object
gps_code             object
iata_code            object
local_code           object
home_link            object
wikipedia_link       object
keywords             object
score                object
last_updated         object
city_region          object
airport_size         object
dtype: object